### IMPORTS

In [167]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import Counter
import pprint
import sys
import spotipy.util as util

### GLOBAL VARIABLES

In [168]:
# users = ["sedepa", "zltm3cqxt0u0s9k4e8oy564dk", "m11oiwplfr6aqk5svov48b40w"]

# sedepa//zltm3cqxt0u0s9k4e8oy564dk

client_ID = "bef34a7d63ba4ace8e7dc8c05de24591"
client_secret = "06764824a5554247a771fedde8889023"

host = "m11oiwplfr6aqk5svov48b40w"

In [169]:
sp = spotipy.Spotify(
    client_credentials_manager = SpotifyClientCredentials(
        "{}".format(client_ID), "{}".format(client_secret)))

genres = []

users = input("Enter the members' Spotify usernames, separated by '//' ").split("//")
print(users)

Enter the members' Spotify usernames, separated by '//' sedepa//zltm3cqxt0u0s9k4e8oy564dk
['sedepa', 'zltm3cqxt0u0s9k4e8oy564dk']


### OBTAIN USERS' PLAYLISTS

In [170]:
def get_user_playlists():
    
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    
    for user in users:
        playlists = sp.user_playlists(user)
        playlists_names = []
        users_playlists_names_IDs[user] = {}
        
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
            
        users_playlists_names[user] = playlists_names
        
    
    mygroup = {"users": {}}
    for user in users:
        mygroup["users"][user] = {} 
        
        
        
    for user in users:
        mygroup["users"][user]["playlists"] = input(
            """
            ·················································
            
            {} , enter the Spotify playlists you would like to upload, separated by '//'. 
                        
            These are all your playlists: 
    
            {} 
            
            
            If you want to submit all, just write "all"
                        
            """
            .format(user, users_playlists_names[user])).split("//")
    
        mygroup["users"][user]["genres"] = input(
            """
            ·················································
            
            {} , now select the genres of the songs for your new playlist.
                        
            You can select out of the following:
    
            {} 
            
            
            If you don't want to filter by genre, just write "any"
            
            Again, separate by '//'
                        
            """
            .format(user, genres)).split("//")


    mygroup["new_playlist_name"] = input(
            """
            ·················································
            
            And finally, write the name for your new playlist :D            
            
            """
            )
    
        
    return users_playlists_names_IDs, mygroup

### OBTAIN TRACKS INFO

In [171]:
def get_tracks_info(users_playlists_names_IDs, mygroup):
    users_playlists_tracks = {}
    for user in mygroup["users"].keys():
        tracks_info = []
        
        if mygroup["users"][user]["playlists"] == ["all"]:
            for playlist_ID in users_playlists_names_IDs[user].values():
                
                tracks_info.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
        
        else:
            for playlist in mygroup["users"][user]["playlists"]:
                tracks_info.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))
    
        users_playlists_tracks[user] = tracks_info
        
    return users_playlists_tracks, mygroup

### OBTAIN FINAL TRACKS

In [172]:
def get_final_tracks(users_playlists_tracks, mygroup):
    
    final_tracks = {}
    
    final_track_names = []
    final_track_IDs = []
    final_track_artists = []

    for user in users_playlists_tracks:
        for playlist in users_playlists_tracks[user]:
            for track in playlist["items"]:
                artist_id = track["track"]["album"]["artists"][0]["external_urls"]["spotify"][32:]
                artist_info = sp.artist(artist_id)
                genre_in_artist = 0
                if mygroup["users"][user]["genres"] == ["any"]:
                    final_track_names.append(track["track"]["name"])
                    final_track_IDs.append(track["track"]["id"])
                    final_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                else:
                    for genre in mygroup["users"][user]["genres"]:
                        if genre in artist_info["genres"]:
                            genre_in_artist += 1
                    if genre_in_artist > 0:
                        final_track_names.append(track["track"]["name"])
                        final_track_IDs.append(track["track"]["id"]) 
                        final_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                    
    
    for i in range(len(final_track_names)):
        final_tracks[final_track_IDs[i]] = [final_track_names[i], final_track_artists[i]]
        
    final_track_IDs = [item for items, c in Counter(final_track_IDs).most_common() for item in [items] * c]
    
    final_track_IDs_unique = []
    
    for track_ID in final_track_IDs:
        if track_ID not in final_track_IDs_unique:
            final_track_IDs_unique.append(track_ID)
    
    return final_tracks, final_track_IDs_unique, final_track_IDs, mygroup

### CREATE DATAFRAME FOR DJ

In [173]:
def create_dataframe(final_tracks, final_track_IDs_unique, final_track_IDs, mygroup):

    tracks = []
    artists = []

    for ID in final_track_IDs:
        tracks.append(final_tracks.get(ID)[0])
        artists.append(final_tracks.get(ID)[1])

    final_tracks_df = pd.DataFrame(data = [final_track_IDs, tracks, artists],
                                   columns = range(len(final_track_IDs)), 
                                   index = ["ID", "Track", "Artist"]).T
    
    display(final_tracks_df)

    return final_track_IDs_unique, mygroup

### CREATE PLAYLIST ON SPOTIFY

In [174]:
def create_playlist(final_track_IDs_unique, mygroup):

    token = util.prompt_for_user_token(host,
                               "playlist-modify-public",
                               client_id = "{}".format(client_ID),
                               client_secret = "{}".format(client_secret),
                               redirect_uri = "https://example.com/callback/")

    sp = spotipy.Spotify(auth = token)
    final_playlist = sp.user_playlist_create(host, mygroup["new_playlist_name"])

    sp.user_playlist_add_tracks(host, final_playlist["uri"][17:], final_track_IDs_unique[:100])

    final_playlist_url = final_playlist["external_urls"]["spotify"]
    
    print("""
          Here's your new combined playlist: 
          
          {}
          
          ENJOY!
          """.format(final_playlist["external_urls"]["spotify"])
          )
    

### PIPELINE

In [175]:
if __name__ == '__main__':
    
    users_playlists_names_IDs, mygroup = get_user_playlists()
    
    users_playlists_tracks, mygroup = get_tracks_info(users_playlists_names_IDs, mygroup)
    
    final_tracks, final_track_IDs_unique, final_track_IDs, mygroup = get_final_tracks(users_playlists_tracks, mygroup)
    
    final_track_IDs_unique, mygroup = create_dataframe(
        final_tracks, final_track_IDs_unique, final_track_IDs, mygroup)
    
    create_playlist(final_track_IDs_unique, mygroup)
    


            ·················································
            
            sedepa , enter the Spotify playlists you would like to upload, separated by '//'. 
                        
            These are all your playlists: 
    
            ['MAD', 'Sunlight', 'Mana Maoli', 'Talking', 'Chosen one', 'Grandes maestros alemanes del arpa que nacieron en el siglo XX', 'Beatz', 'Interference', 'Movida Milennial', 'Si algún día me convierto en jugador de NBA estos serán mis jams', 'Al Son Club Social', 'Night-light', 'Bonfire.', 'El gusto es mio', 'Palante', 'Wgłębienie', 'Nueva barrica', 'Ace à Paris', 'Chill', "Pugs 'n' hugs", 'Just de Paso', 'Tobillos', 'Niggas Flow', 'retro touch', 'Sounds great', 'Coast Road', 'New Zihuatanejo', 'Mudarse a Zihuatanejo', 'Dormir', 'Baby you and me', 'De ayer y de hoy', 'Más Eñe', 'A little of verguenza, please', 'Correr'] 
            
            
            If you want to submit all, just write "all"
                        
            

,ID,Track,Artist
0,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
1,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
2,7CJGFIChxuwIAQb5GbqfRF,Tuolumne,Steezmonks
3,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
4,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
5,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
6,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
7,4aAfLSx9IthpC3Pw5pNk3E,Como Si Fueras a Morir Mañana,Leiva
8,4aAfLSx9IthpC3Pw5pNk3E,Como Si Fueras a Morir Mañana,Leiva
9,2a2MWHc7gtuecUWU9WIasb,Expertos,Leiva



          Here's your new combined playlist: 
          
          https://open.spotify.com/playlist/2tjOr74T37x7JCZuSB0mXD
          
          ENJOY!
          
